# AzureAISearchRetriever

[Azure AI Search](https://learn.microsoft.com/azure/search/search-what-is-azure-search)（以前称为 `Azure Cognitive Search`）是一项微软云搜索服务，为开发人员提供了大规模信息检索的基础设施、API 和工具，用于矢量、关键字和混合查询。

`AzureAISearchRetriever` 是一个集成模块，用于从非结构化查询中返回文档。它基于 BaseRetriever 类，并且以 Azure AI Search 的 2023-11-01 稳定版 REST API 为目标，这意味着它支持矢量索引和查询。

本指南将帮助你开始使用 Azure AI Search [检索器](/docs/concepts/retrievers)。有关 `AzureAISearchRetriever` 所有功能和配置的详细文档，请参阅 [API 参考](https://python.langchain.com/api_reference/community/retrievers/langchain_community.retrievers.azure_ai_search.AzureAISearchRetriever.html)。

`AzureAISearchRetriever` 取代了即将弃用的 `AzureCognitiveSearchRetriever`。我们建议切换到基于最新稳定版搜索 API 的新版本。

### 集成详情

import {ItemTable} from "@theme/FeatureTables";

<ItemTable category="document_retrievers" item="AzureAISearchRetriever" />


## 设置

要使用此模块，你需要：

+ 一个 Azure AI Search 服务。如果你注册 Azure 试用版，可以[免费创建一个](https://learn.microsoft.com/azure/search/search-create-service-portal)。免费服务有较低的配额，但足以运行此笔记本中的代码。

+ 一个具有矢量字段的现有索引。有几种方法可以创建一个，包括使用 [矢量存储模块](../vectorstores/azuresearch.ipynb)。或者，[尝试 Azure AI Search REST API](https://learn.microsoft.com/azure/search/search-get-started-vector)。

+ 一个 API 密钥或 Azure AD 令牌。
  + API 密钥在创建搜索服务时生成。如果你只是查询索引，可以使用查询 API 密钥；否则，请使用管理员 API 密钥。有关详细信息，请参阅[查找你的 API 密钥](https://learn.microsoft.com/azure/search/search-security-api-keys?tabs=rest-use%2Cportal-find%2Cportal-query#find-existing-keys)。
  + Azure AD 令牌可以与 Azure 托管标识一起使用。有关详细信息，请参阅[使用标识将你的应用连接到 Azure AI Search](https://learn.microsoft.com/en-us/azure/search/keyless-connections?tabs=python%2Cazure-cli)。

然后，我们可以将搜索服务名称、索引名称和 API 密钥设置为环境变量（或者，你可以将它们作为参数传递给 `AzureAISearchRetriever`）。搜索索引提供可搜索的内容。

使用 API 密钥

In [ ]:
import os

os.environ["AZURE_AI_SEARCH_SERVICE_NAME"] = "<YOUR_SEARCH_SERVICE_NAME>"
os.environ["AZURE_AI_SEARCH_INDEX_NAME"] = "<YOUR_SEARCH_INDEX_NAME>"
os.environ["AZURE_AI_SEARCH_API_KEY"] = "<YOUR_API_KEY>"

带有 Azure AD 令牌

In [ ]:
import os

os.environ["AZURE_AI_SEARCH_SERVICE_NAME"] = "<YOUR_SEARCH_SERVICE_NAME>"
os.environ["AZURE_AI_SEARCH_INDEX_NAME"] = "<YOUR_SEARCH_INDEX_NAME>"
os.environ["AZURE_AI_SEARCH_AD_TOKEN"] = "<YOUR_AZURE_AD_TOKEN>"

如果你想从单个查询中获得自动化追踪，你也可以通过取消下面一行的注释来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

这个检索器位于 `langchain-community` 包中。我们还需要一些额外的依赖项：

In [ ]:
%pip install --upgrade --quiet langchain-community
%pip install --upgrade --quiet langchain-openai
%pip install --upgrade --quiet  azure-search-documents>=11.4
%pip install --upgrade --quiet  azure-identity

## 实例化

对于 `AzureAISearchRetriever`，请提供 `index_name`、`content_key` 以及设置您希望检索结果数量的 `top_k`。将 `top_k` 设置为零（默认值）将返回所有结果。

In [ ]:
from langchain_community.retrievers import AzureAISearchRetriever

retriever = AzureAISearchRetriever(
    content_key="content", top_k=1, index_name="langchain-vector-demo"
)

## 用法

现在您可以使用它从 Azure AI Search 中检索文档。
这是您用于此目的调用的方法。它将返回与查询相关的所有文档。

In [ ]:
retriever.invoke("here is my unstructured query string")

## 示例

本节演示如何在内置的示例数据上使用检索器。如果您已经在搜索服务上拥有向量索引，则可以跳过此步骤。

首先提供终结点和密钥。由于在此步骤中我们将创建向量索引，因此请指定文本嵌入模型以获取文本的向量表示。此示例假定您使用的是 Azure OpenAI 和 text-embedding-ada-002 的部署。因为此步骤会创建索引，请确保为您的搜索服务使用管理员 API 密钥。

In [ ]:
import os

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_community.vectorstores import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_text_splitters import TokenTextSplitter

os.environ["AZURE_AI_SEARCH_SERVICE_NAME"] = "<YOUR_SEARCH_SERVICE_NAME>"
os.environ["AZURE_AI_SEARCH_INDEX_NAME"] = "langchain-vector-demo"
os.environ["AZURE_AI_SEARCH_API_KEY"] = "<YOUR_SEARCH_SERVICE_ADMIN_API_KEY>"
azure_endpoint: str = "<YOUR_AZURE_OPENAI_ENDPOINT>"
azure_openai_api_key: str = "<YOUR_AZURE_OPENAI_API_KEY>"
azure_openai_api_version: str = "2023-05-15"
azure_deployment: str = "text-embedding-ada-002"

我们将使用 Azure OpenAI 的 embedding 模型将文档转换为存储在 Azure AI Search 向量存储中的 embedding。我们还将索引名称设置为 `langchain-vector-demo`。这将创建一个与该索引名称关联的新向量存储。

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    model=azure_deployment,
    azure_endpoint=azure_endpoint,
    openai_api_key=azure_openai_api_key,
)

vector_store: AzureSearch = AzureSearch(
    embedding_function=embeddings.embed_query,
    azure_search_endpoint=os.getenv("AZURE_AI_SEARCH_SERVICE_NAME"),
    azure_search_key=os.getenv("AZURE_AI_SEARCH_API_KEY"),
    index_name="langchain-vector-demo",
)

接下来，我们将数据加载到新创建的向量存储中。在此示例中，我们加载 `state_of_the_union.txt` 文件。我们将文本分割成 400 个 token 的块，不重叠。最后，将文档作为嵌入添加到来向量存储中。

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../../how_to/state_of_the_union.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

接下来，我们将创建一个检索器。当前的 `index_name` 变量是上一步的 `langchain-vector-demo`。如果您跳过了向量存储的创建，请在参数中提供您的索引名称。在此查询中，将返回排名第一的结果。

In [21]:
retriever = AzureAISearchRetriever(
    content_key="content", top_k=1, index_name="langchain-vector-demo"
)

现在我们可以从上传的文档中检索与我们的查询相关的数据。

In [ ]:
retriever.invoke("does the president have a plan for covid-19?")

## 在链式调用中使用

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)

llm = ChatOpenAI(model="gpt-4o-mini")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("does the president have a plan for covid-19?")

## API 参考

有关 `AzureAISearchRetriever` 所有特性和配置的详细文档，请前往 [API 参考](https://python.langchain.com/api_reference/community/retrievers/langchain_community.retrievers.azure_ai_search.AzureAISearchRetriever.html)。